In [1]:
import regex as re
import pandas as pd

In [4]:
sample_game = pd.read_excel('Outline_And_Sample.xlsx.xlsx',
                             sheet_name='Sample',
                             skiprows=1,
                             usecols=['TIME','TEAM','PLAY','SCORE','Prev State','New State'])

In [6]:
sample_game['TEAM']=sample_game['TEAM'].str.replace(' title=','')

In [17]:
sample_game=sample_game.fillna('')

In [8]:
team_A = 'UT'
team_B = 'LSU'

In [20]:
new_states = []
for idx in range(len(sample_game)):
    new_state='new_state'
    current_play = sample_game['PLAY'][idx]
    current_state = sample_game['Prev State'][idx]
    #logic for "UNNEC" rows
    if "MISSED" in current_play:
        new_state='UNNEC'
    elif "STEAL" in current_play:
        new_state='UNNEC'
    elif 'Assist' in current_play:
        new_state='UNNEC'
    elif 'Block' in current_play:
        new_state='UNNEC'
    elif "Subbing" in current_play:
        new_state='UNNEC'
    elif "timeout" in current_play:
        new_state='UNNEC'

    else:
        if team_A in current_play:

            #Logic for Fouls
            if 'Foul' in current_play:
                #if foul happens, we need the next play to determine what the result is
                #Will need to amend if the shot is GOOD before the foul?
                #Confirmed - if shot is GOOD before the foul, then foul results in UNNEC and previous line results in f2 or f3
                prev_play='Filler String'
                try:
                    prev_play = sample_game['PLAY'][idx-1]
                except KeyError:
                    pass

                if 'GOOD' in prev_play:
                    new_state = 'UNNEC'
                else:
                    next_play = sample_game['PLAY'][idx+1]
                    if 'Free Throw' in next_play:
                        new_state = 'Bf0'
                    else:
                        new_state = 'Bi0'

            #Logic for MADE free throws
            elif "Free Throw GOOD" in current_play:
                #if free throw is made, we need the next play to determine if next play is FT or inbound
                #NOTE - THIS WILL BE DIFFERENT WITH TECHNICAL FOULS
                next_play = sample_game['PLAY'][idx+1]
                if 'Free Throw' in next_play:
                    new_state = 'Af1'
                else:
                    new_state = 'Bi1'

            #Logic for "REBOUND"
            elif "REBOUND" in current_play:
                #if rebound on DEAD BALL, this transitions to same team FT
                if 'dead ball' in current_play:
                    new_state = 'Af0'
                else:
                    new_state = 'Ar0'

            #Logic for "TURNOVER"
            elif "TURNOEVER" in current_play:
                new_state='Ar0'

            #Logic for "GOOD" shots
            elif "GOOD" in current_play:
                #determine point value
                pv=0
                if 'Jumper' in current_play:
                    pv=2
                elif '3 Pointer' in current_play:
                    pv=3
                elif 'Layup' in current_play:
                    pv=2
                elif 'Dunk' in current_play:
                    pv=2
                #Need to check next play for FOUL
                next_play = sample_game['PLAY'][idx+1]
                if 'Foul' in next_play:
                    new_state = 'Af{}'.format(str(pv))
                else:
                    new_state = 'Bi{}'.format(str(pv))
            else:
                new_state = 'Team A Logic Missed'




        elif team_B in current_play:

            #Logic for Fouls
            if 'Foul' in current_play:
                #if foul happens, we need the next play to determine what the result is
                #Will need to amend if the shot is GOOD before the foul?
                #Confirmed - if shot is GOOD before the foul, then foul results in UNNEC and previous line results in f2 or f3
                prev_play='Filler String'
                try:
                    prev_play = sample_game['PLAY'][idx-1]
                except KeyError:
                    pass

                if 'GOOD' in prev_play:
                    new_state = 'UNNEC'
                else:
                    next_play = sample_game['PLAY'][idx+1]
                    if 'Free Throw' in next_play:
                        new_state = 'Af0'
                    else:
                        new_state = 'Ai0'

            #Logic for MADE free throws
            elif "Free Throw GOOD" in current_play:
                #if free throw is made, we need the next play to determine if next play is FT or inbound
                #NOTE - THIS WILL BE DIFFERENT WITH TECHNICAL FOULS
                next_play = sample_game['PLAY'][idx+1]
                if 'Free Throw' in next_play:
                    new_state = 'Bf1'
                else:
                    new_state = 'Ai1'

            #Logic for "REBOUND"
            elif "REBOUND" in current_play:
                #if rebound on DEAD BALL, this transitions to same team FT
                if 'dead ball' in current_play:
                    new_state = 'Bf0'
                else:
                    new_state = 'Br0'

            #Logic for "TURNOVER"
            elif "TURNOEVER" in current_play:
                new_state='Br0'

            #Logic for "GOOD" shots
            elif "GOOD" in current_play:
                #determine point value
                pv=0
                if 'Jumper' in current_play:
                    pv=2
                elif '3 Pointer' in current_play:
                    pv=3
                elif 'Layup' in current_play:
                    pv=2
                elif 'Dunk' in current_play:
                    pv=2
                #Need to check next play for FOUL
                next_play = sample_game['PLAY'][idx+1]
                if 'Foul' in next_play:
                    new_state = 'Bf{}'.format(str(pv))
                else:
                    new_state = 'Ai{}'.format(str(pv))

            else:
                new_state = 'Team B logic missed'
        
        else:
            new_state = "all logic missed"
    new_states.append(new_state)

In [21]:
new_states

['Af0',
 'Af1',
 'Bi1',
 'UNNEC',
 'Ar0',
 'Bi3',
 'UNNEC',
 'UNNEC',
 'Br0',
 'UNNEC',
 'Ar0',
 'Bi3',
 'UNNEC',
 'UNNEC',
 'Ar0',
 'Bi2',
 'UNNEC',
 'Team B logic missed',
 'UNNEC',
 'UNNEC',
 'Ar0',
 'Bi3',
 'UNNEC',
 'UNNEC',
 'Br0',
 'Bf0',
 'UNNEC',
 'Bf0',
 'UNNEC',
 'UNNEC',
 'Ai1',
 'Bi2',
 'UNNEC',
 'UNNEC',
 'Br0',
 'Ai2',
 'UNNEC',
 'Br0',
 'Bi0',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'Br0',
 'UNNEC',
 'Br0',
 'Ai2',
 'Team A Logic Missed',
 'Team B logic missed',
 'Team A Logic Missed',
 'Bi2',
 'UNNEC',
 'Team B logic missed',
 'Team A Logic Missed',
 'UNNEC',
 'Br0',
 'UNNEC',
 'Ar0',
 'UNNEC',
 'Br0',
 'Ai3',
 'UNNEC',
 'UNNEC',
 'Br0',
 'Ai2',
 'Bi2',
 'UNNEC',
 'Ai2',
 'Bi2',
 'Ai3',
 'UNNEC',
 'UNNEC',
 'Af0',
 'Ai0',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'UNNEC',
 'Bi2',
 'UNNEC',
 'Team B logic missed'